In [122]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

In [97]:
train_data = pd.read_csv('train_2.txt', sep = '\t')
test_data = pd.read_csv('test_2.txt', sep = '\t')

In [98]:
train_data.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [99]:
test_data.head()

,id,name,description
0,200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
1,200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
2,200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
4,200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


## Baseline submission

Для baseline будем использовать только наименование вакансии

In [100]:
# train, test
train_df = train_data.drop(labels = ['id', 'description', 'target'], axis = 1)
y = train_data['target']
test_df = test_data.drop(labels = ['id', 'description'], axis = 1)

In [101]:
# Выделим выборку под кросс-валидацию и holdout выборку
X_train, X_holdout, y_train, y_holdout = train_test_split(train_df, y, test_size=0.3, random_state=42,
                                                         shuffle = True, stratify = y)

### Fit

In [120]:
# pipeline
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [121]:
for clf in [LogisticRegression, SGDClassifier, RandomForestClassifier, GradientBoostingClassifier]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(), TfidfTransformer(), clf()), 
                           list(X_train['name'].values), list(y_train.values), scoring = 'roc_auc', cv = 5).mean())
    print("\n")

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.9823728605641777


<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>


C:\Users\temirlan.smail\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\temirlan.smail\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e

0.9773303615553111


<class 'sklearn.ensemble.forest.RandomForestClassifier'>
0.98358881573573


<class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
0.9557007434655411




Лучшая модель на кросс-валдидации это Случайный лес. Применим случайный лес к отложенной выборке и посмотрим на ROC AUC

In [123]:
rf_text_clf_pip = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', RandomForestClassifier(random_state = 42)),
])

In [124]:
rf_text_clf_pip.fit(list(X_train['name'].values), list(y_train.values))

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [126]:
y_pred = rf_text_clf_pip.predict(list(X_holdout['name'].values))
print('ROC AUC: {}'.format(roc_auc_score(y_holdout, y_pred)))

ROC AUC: 0.9545458118845204


Обучим rf_text_clf_pip на всей тренировочной выборке: train_df, y

In [127]:
rf_text_clf_pip.fit(list(train_df['name'].values), list(y.values))

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

### Predict and Submit

In [133]:
# функция записи файла отправки
def write_submission_file(ids, targets, filename):
    submission = pd.DataFrame(data = ids, columns =['id'])
    submission['target'] = targets
    submission.to_csv(filename, sep = ',', header = True, index = False)

In [132]:
ids = test_data['id'].values
y_test_pred = rf_text_clf_pip.predict(list(test_df['name'].values))
write_submission_file(ids, y_test_pred, 'baseline_sub_0.csv')

       id  target
0  200000       1
1  200001       1
2  200002       1
3  200003       1
4  200004       0
